In [91]:
# Week 2: Cleaning TMDb Dataset (Handling Missing Values, Outliers, and Duplicates)

# This notebook contains all Python-based cleaning steps derived from insights gathered during the OpenRefine exploration.
# Please review the OpenRefine documentation beforehand, as it thoroughly explains the visual findings that guide our cleaning process.
# OpenRefine was used to visually identify inconsistencies, formatting issues, and potential data discrepancies.
# Python is used here to carry out precise and reproducible data cleaning for the TMDb dataset.
# A separate notebook contains the IMDB data cleaning process, the two datasets were divided for clarity and ease of visibility.

In [92]:
#import functions

import requests
import pandas as pd
from datetime import datetime
import json
import os
import hashlib
from pathlib import Path
import numpy as np
import gzip
import io


In [93]:
#confirm raw dataset is read in properly

tmdb = pd.read_csv('tmdb_5000_movies.csv')

In [94]:
tmdb.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [95]:
# Check data types, missing values, and overall info
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [96]:
# Look for duplicate rows (just to confirm none slipped in)
tmdb.duplicated().sum()

0

In [97]:
# Drop columns that are not relevant to the project goals
cols_to_drop = [
    'homepage', 'id', 'keywords', 'original_language', 'overview', 
    'production_companies', 'production_countries', 'spoken_languages', 
    'status', 'tagline', 'original_title'
]
tmdb.drop(columns=cols_to_drop, inplace=True)

In [98]:
# Confirm column removal
tmdb.head()

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577,2009-12-10,2787965087,162.0,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",139.082615,2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",107.376788,2015-10-26,880674609,148.0,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",112.312950,2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",43.926995,2012-03-07,284139100,132.0,John Carter,6.1,2124


In [99]:
# Ensure 'budget' is numeric and convert to millions to match IMDb
tmdb['budget'] = pd.to_numeric(tmdb['budget'], errors='coerce') / 1000000
tmdb.rename(columns={'budget': 'budget_in_millions'}, inplace=True)

In [100]:
# Drop rows with zero budget (unreleased movies)
tmdb = tmdb[tmdb['budget_in_millions'] > 0]

In [101]:
# Confirm 'budget'

tmdb.head(10)

,budget_in_millions,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,237.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577,2009-12-10,2787965087,162.0,Avatar,7.2,11800
1,300.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",139.082615,2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500
2,245.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",107.376788,2015-10-26,880674609,148.0,Spectre,6.3,4466
3,250.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",112.312950,2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106
4,260.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",43.926995,2012-03-07,284139100,132.0,John Carter,6.1,2124
5,258.0,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",115.699814,2007-05-01,890871626,139.0,Spider-Man 3,5.9,3576
6,260.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",48.681969,2010-11-24,591794936,100.0,Tangled,7.4,3330
7,280.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",134.279229,2015-04-22,1405403694,141.0,Avengers: Age of Ultron,7.3,6767
8,250.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",98.885637,2009-07-07,933959197,153.0,Harry Potter and the Half-Blood Prince,7.4,5293
9,250.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",155.790452,2016-03-23,873260194,151.0,Batman v Superman: Dawn of Justice,5.7,7004


In [102]:
# TMDB genres column is complex JSON-like string that is quite confusing. Here I am extracting just the genre names, alphabetizing them, and separating them by commas to match the IMDB data set

def parse_genres(genre_str):
    if pd.isna(genre_str) or genre_str == '[]':
        return np.nan
    try:
        genre_list = [d['name'] for d in ast.literal_eval(genre_str)]
        genre_list.sort()
        return ', '.join(genre_list)
    except:
        return np.nan

# I will apply parsing function to get rid of unecessary characters
tmdb['genre'] = tmdb['genres'].apply(parse_genres)

# Drop the original 'genres' column
tmdb.drop(columns=['genres'], inplace=True)

# Drop rows where genre is missing to avoid discrepencies in our future analysis
tmdb = tmdb.dropna(subset=['genre'])

# Confirm the result
tmdb[['genre']].head()

,genre
0,"Action, Adventure, Fantasy, Science Fiction"
1,"Action, Adventure, Fantasy"
2,"Action, Adventure, Crime"
3,"Action, Crime, Drama, Thriller"
4,"Action, Adventure, Science Fiction"


In [103]:
#Convert popularity to be read as a number, not a string
tmdb['popularity'] = pd.to_numeric(tmdb['popularity'], errors='coerce')

# Drop outliers / missing values
tmdb = tmdb[tmdb['popularity'] > 0]

In [104]:
# Extract year from 'release_date' and convert to integer
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'], errors='coerce')
tmdb['release_year'] = tmdb['release_date'].dt.year.astype('Int64')

# Drop rows where release_year is missing
tmdb = tmdb.dropna(subset=['release_year'])

# Drop the original release_date column
tmdb.drop(columns=['release_date'], inplace=True)

In [105]:
# Convert revenue to millions, drop zeros
tmdb['revenue'] = pd.to_numeric(tmdb['revenue'], errors='coerce') / 1000000
tmdb.rename(columns={'revenue': 'revenue_in_millions'}, inplace=True)
tmdb = tmdb[tmdb['revenue_in_millions'] > 0]

In [106]:
# Ensure runtime is numeric, drop zero or missing runtimes
tmdb['runtime'] = pd.to_numeric(tmdb['runtime'], errors='coerce')
tmdb.rename(columns={'runtime': 'runtime_in_minutes'}, inplace=True)
tmdb = tmdb[tmdb['runtime_in_minutes'] > 0]

In [112]:
# Ensure titles are unique
tmdb.duplicated(subset='title').sum()
# Two titles were the same, so I confirmed that they are indeed different, no further action is needed, as they are unique.
tmdb[tmdb.duplicated(subset='title', keep=False)].sort_values('title')

,budget_in_millions,popularity,revenue_in_millions,runtime_in_minutes,title,vote_average,vote_count,genre,release_year
972,44.0,42.933027,63.327201,125.0,The Host,6.0,1817,"Action, Adventure, Romance, Science Fiction, T...",2013
2877,11.0,27.655270,88.489643,119.0,The Host,6.7,537,"Drama, Horror, Science Fiction",2006


In [108]:
#Verify Vote Average is read correctly

tmdb['vote_average'] = pd.to_numeric(tmdb['vote_average'], errors='coerce')
tmdb = tmdb[tmdb['vote_average'] > 0]

In [109]:
# Confirm the count is read as a number, not a string
tmdb['vote_count'] = pd.to_numeric(tmdb['vote_count'], errors='coerce')
tmdb = tmdb[tmdb['vote_count'] > 10]  # Drop low-sample movies to avoid skew in later analysis


In [110]:
#Confirm everything looks correct
tmdb.info()
tmdb.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3177 entries, 0 to 4798
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   budget_in_millions   3177 non-null   float64
 1   popularity           3177 non-null   float64
 2   revenue_in_millions  3177 non-null   float64
 3   runtime_in_minutes   3177 non-null   float64
 4   title                3177 non-null   object 
 5   vote_average         3177 non-null   float64
 6   vote_count           3177 non-null   int64  
 7   genre                3177 non-null   object 
 8   release_year         3177 non-null   Int64  
dtypes: Int64(1), float64(5), int64(1), object(2)
memory usage: 251.3+ KB


,budget_in_millions,popularity,revenue_in_millions,runtime_in_minutes,title,vote_average,vote_count,genre,release_year
0,237.0,150.437577,2787.965087,162.0,Avatar,7.2,11800,"Action, Adventure, Fantasy, Science Fiction",2009
1,300.0,139.082615,961.000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,"Action, Adventure, Fantasy",2007
2,245.0,107.376788,880.674609,148.0,Spectre,6.3,4466,"Action, Adventure, Crime",2015
3,250.0,112.312950,1084.939099,165.0,The Dark Knight Rises,7.6,9106,"Action, Crime, Drama, Thriller",2012
4,260.0,43.926995,284.139100,132.0,John Carter,6.1,2124,"Action, Adventure, Science Fiction",2012


In [113]:
tmdb.to_csv('tmdb_cleaned.csv', index=False)